In [1]:
from argparse import Namespace
import collections
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import torch
import pprint

In [2]:
def set_seeds(seed,cuda):
    np.random.seed(seed)# when use np.random.rand() or np.random.randn()
    torch.manual_seed(seed)# when use torch.rand() or torch.randn()
    if cuda:
        torch.cuda.manual_seed_all(seed)

In [3]:
args = Namespace(
    seed = 999,
    cuda = True,
    shuffle = True,
    data_file = "tumors.csv",
    reduced_data_file = "tumors_reduced.csv",
    train_size = 0.75,
    test_size = 0.25,
    num_hidden_units = 100,
    learning_rate = 1e-3,
    num_epochs = 100,
)

In [4]:
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")

In [5]:
set_seeds(seed = args.seed,cuda=args.cuda)
pprint.pprint(vars(args),indent=4)
print(args.device)

{   'cuda': True,
    'data_file': 'tumors.csv',
    'device': device(type='cuda'),
    'learning_rate': 0.001,
    'num_epochs': 100,
    'num_hidden_units': 100,
    'reduced_data_file': 'tumors_reduced.csv',
    'seed': 999,
    'shuffle': True,
    'test_size': 0.25,
    'train_size': 0.75}
cuda


In [7]:
import re
import urllib